In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import joblib
from IPython.display import clear_output
import matplotlib.pyplot as plt
from torch.autograd import Variable

import utils


In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
inputs = joblib.load('../Vectors/aluminium_vectors')
labels = joblib.load('../Vectors/aluminium_labels')
dates = joblib.load('../Vectors/dates')

In [47]:
print(inputs.shape)
print(labels.shape)
print(dates.shape)

(969, 7, 30)
(969,)
(969,)


In [48]:
train_batches,validation_batches,test_batches = utils.train_test_split(inputs,labels,dates,inputs.shape[0],0.8,0)

In [49]:
train_data,train_labels,train_dates = train_batches[0]
test_data,test_labels,test_dates = test_batches[0]
val_data,val_labels,val_dates = validation_batches[0]
print("Train Dates: " + train_dates[0] + " " + train_dates[-1])
print("Val Dates: " + val_dates[0] + " " + val_dates[-1])
print("Test Dates: " + test_dates[0] + " " + test_dates[-1])

Train Dates: 2017-05-25 2020-05-13
Val Dates: 2020-05-14 2020-09-25
Test Dates: 2020-09-28 2021-02-09


In [50]:
train_data = torch.from_numpy(train_data).to(dtype=torch.float16).to(device)
train_labels = torch.from_numpy(train_labels).to(device)
test_data = torch.from_numpy(test_data).to(dtype=torch.float16).to(device)
test_labels = torch.from_numpy(test_labels).to(device)
val_data = torch.from_numpy(val_data).to(dtype=torch.float16).to(device)
val_labels = torch.from_numpy(val_labels).to(device)

In [52]:
train_labels = torch.reshape(train_labels, (train_labels.shape[0], 1))
test_labels = torch.reshape(test_labels, (test_labels.shape[0], 1))
val_labels = torch.reshape(val_labels, (val_labels.shape[0], 1))

In [53]:
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)
print(val_data.shape)
print(val_labels.shape)

torch.Size([775, 7, 30])
torch.Size([775, 1])
torch.Size([97, 7, 30])
torch.Size([97, 1])
torch.Size([97, 7, 30])
torch.Size([97, 1])


In [54]:
## define a dictionary with the parameters of the Neural Network
params_dictionary = {}
params_dictionary['embedding_size'] = train_data[0].shape[1]
params_dictionary['hidden_dim'] = 256
params_dictionary['linear_dims'] = [64,32]
params_dictionary['bidirectional'] = True
params_dictionary['label_size'] = 1
params_dictionary['dropout'] = 0.3
params_dictionary['rnn_layers_num'] = 2
params_dictionary['attention_layer'] = False

In [55]:
class Attention(nn.Module):
    def __init__(self, device,hidden_size):
        super(Attention, self).__init__()
        self.device = device
        self.hidden_size = hidden_size

        self.concat_linear = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size, hidden_size)

        ## add this model the same same device with the RNN
        self.to(device)

    def forward(self, rnn_outputs, final_hidden_state):
        attn_weights = self.attn(rnn_outputs) # (batch_size, seq_len, hidden_dim)
        attn_weights = torch.bmm(attn_weights, final_hidden_state.unsqueeze(2))
        attn_weights = F.softmax(attn_weights.squeeze(2), dim=1)
        context = torch.bmm(rnn_outputs.transpose(1, 2), attn_weights.unsqueeze(2)).squeeze(2)
        attn_hidden = torch.tanh(self.concat_linear(torch.cat((context, final_hidden_state), dim=1)))
        return attn_hidden, attn_weights

In [56]:
class RnnRegressor(nn.Module):
    def __init__(self, device, params_dictionary):
        super(RnnRegressor, self).__init__()
        self.params = params_dictionary
        self.device = device

        # Calculate number of directions
        self.num_directions = 2 if self.params.get('bidirectional') == True else 1

        # define an attention model
        # Choose attention model
        self.attention = Attention(self.device,self.params.get('hidden_dim')* self.num_directions)

        ## here store in a list all the dimensions of the layers rnn_output --> linear layers --> labels layer
        self.linear_dims = [self.params.get('hidden_dim') * self.num_directions] + self.params.get('linear_dims')
        self.linear_dims.append(self.params.get('label_size'))

        # Work with LSTM cell for now
        self.rnn = nn.LSTM

        ## define the RNN layer
        self.rnn = self.rnn(self.params.get('embedding_size'),
                            self.params.get('hidden_dim'),
                            num_layers=self.params.get('rnn_layers_num'),
                            bidirectional=self.params.get('bidirectional'),
                            dropout=float(self.params.get('dropout')),
                            batch_first=True)
        
        ## the hidden state of the RNN empty for now
        self.hidden = None
        
        # Define set of fully connected layers (Linear Layer + Activation Layer)
        ## this set of layers takes the output of the RNN or the Attention layer and applies a feedforward NN on it
        ## consecutive linear + Relu layers are applied (the final layer does not have a relu activation!)
        self.linears = nn.ModuleList()
        for i in range(0, len(self.linear_dims)-1):
            if self.params.get('dropout') > 0.0:
                self.linears.append(nn.Dropout(p=self.params.get('dropout')))
            linear_layer = nn.Linear(self.linear_dims[i], self.linear_dims[i+1])
            self.init_weights(linear_layer)
            self.linears.append(linear_layer)
            if i == len(self.linear_dims) - 1:
                break
            self.linears.append(nn.ReLU())

        self.to(self.device)

    def init_weights(self, layer):
        if type(layer) == nn.Linear:
            torch.nn.init.xavier_uniform_(layer.weight)
            layer.bias.data.fill_(0.01)

    def init_hidden(self, batch_size):
        return (torch.zeros(self.params.get('rnn_layers_num') * self.num_directions, batch_size, self.params.get('hidden_dim')).to(self.device),
              torch.zeros(self.params.get('rnn_layers_num') * self.num_directions, batch_size, self.params.get('hidden_dim')).to(self.device))
      

    def forward(self, inputs):
        batch_size, seq_len, embedding_size = inputs.shape


        embedded_inputs = inputs

        ## initialise the hidden state of the RNN
        self.hidden = self.init_hidden(batch_size)

        ## pass the data through the recurrent layer
        rnn_output, self.hidden = self.rnn(embedded_inputs.float(), self.hidden)

        ## Collect last hidden state
        final_state = self.hidden[0].view(self.params.get('rnn_layers_num'), self.num_directions, batch_size, self.params.get('hidden_dim'))[-1]

        # Handle directions if more than one
        final_hidden_state = None
        ## in case we have only one direction
        if self.num_directions == 1:
            final_hidden_state = final_state.squeeze(0)
        ## in case we have 2 directions concatenate these two states
        elif self.num_directions == 2:
            h_1, h_2 = final_state[0], final_state[1]
            final_hidden_state = torch.cat((h_1, h_2), 1)  # Concatenate both states

        ## Attention Layer
        if(self.params.get('attention_layer') == False):
            X = final_hidden_state
        else:
            rnn_output = rnn_output.permute(1, 0, 2)
            X, attention_weights = self.attention(rnn_output, final_hidden_state)

        # Push through linear layers
        for l in self.linears:
            X = l(X)

        return X

In [57]:
model = RnnRegressor(device,params_dictionary)

In [58]:
loss = nn.MSELoss()
optimizer =  torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss = []
valid_loss = []

epochs = 100

for i in range(epochs):
    loss_batches = []
    optimizer.zero_grad()
    y_pred = model.forward(train_data)
    single_loss = loss(y_pred,train_labels.float()).to(device)
    single_loss.backward()
    optimizer.step()
    train_loss.append(single_loss.item())

    ## compute and store the validation loss
    valid_pred = model.forward(val_data)
    valid_loss.append(loss(valid_pred,val_labels.float().to(device)))

    print("epoch  " + str(i+1) + " " + str(train_loss[-1]))
    clear_output(wait=True)

epoch  9 0.7446760535240173
